# 📦 CELLULE 1 : Imports et Configuration


In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              MNIST MLP - Classification de Chiffres Manuscrits           ║
║                                                                          ║
║  Architecture : Multi-Layer Perceptron (Fully Connected)                 ║
║  Objectif : Classifier les chiffres MNIST (0-9)                          ║
║  Techniques : Optimisation bayésienne, Early stopping, Régularisation    ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

# ============================================================================
# IMPORTS
# ============================================================================

# Librairies standards
import numpy as np
import pandas as pd
import json
import time
from pathlib import Path

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-Learn - Métriques
from sklearn.metrics import (
    roc_auc_score, 
    confusion_matrix,
    classification_report, 
    silhouette_score,
    davies_bouldin_score
)

# Réduction de dimensionnalité
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Optimisation bayésienne
import optuna
from optuna.visualization import plot_param_importances, plot_optimization_history

# Réduction de dimensionnalité avancée
try:
    import umap
    print("✅ UMAP disponible")
except ImportError:
    print("⚠️  UMAP non installé - Installer avec: pip install umap-learn")

try:
    import pacmap
    print("✅ PaCMAP disponible")
except ImportError:
    print("⚠️  PaCMAP non installé - Installer avec: pip install pacmap")

# ============================================================================
# CONFIGURATION GLOBALE
# ============================================================================

# Seed pour reproductibilité
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

# Device (GPU si disponible, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n{'='*70}")
print(f"⚙️  CONFIGURATION")
print(f"{'='*70}")
print(f"🖥️  Device : {device}")
print(f"🎲 Random seed : {RANDOM_SEED}")

if device.type == 'cuda':
    print(f"🚀 GPU : {torch.cuda.get_device_name(0)}")
    print(f"💾 Mémoire GPU : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# ============================================================================
# CHEMINS DES FICHIERS
# ============================================================================

# Configuration pour Google Drive (adapter selon ton environnement)
DRIVE_PATH = '/content/drive/MyDrive/La Plateforme/Handwritten Digits'

# Chemins des données
DATA_TRAIN_PATH = f'{DRIVE_PATH}/mnist_train.csv'
DATA_TEST_PATH = f'{DRIVE_PATH}/mnist_test.csv'

# Chemins de sauvegarde
MODEL_PATH = f'{DRIVE_PATH}/model_final_MLP.pth'
HYPERPARAMS_PATH = f'{DRIVE_PATH}/best_hyperparams_MLP.json'
GRAPH_DIR = Path(f'{DRIVE_PATH}/graphiques')

# Créer le dossier de graphiques si nécessaire
GRAPH_DIR.mkdir(exist_ok=True, parents=True)

print(f"\n📂 Chemins configurés :")
print(f"   Train : {DATA_TRAIN_PATH}")
print(f"   Test  : {DATA_TEST_PATH}")
print(f"   Modèle : {MODEL_PATH}")
print(f"   Hyperparamètres : {HYPERPARAMS_PATH}")
print(f"   Graphiques : {GRAPH_DIR}")

print(f"\n{'='*70}")
print("✅ CONFIGURATION TERMINÉE")
print(f"{'='*70}")


# 📊 CELLULE 2 : Chargement et Préparation des Données

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                  CHARGEMENT ET SPLIT DES DONNÉES                         ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

def load_mnist_data(train_path, test_path, train_split=0.8, shuffle=True):
    """
    Charge les données MNIST et effectue un split train/dev/test.
    
    Le dataset original MNIST train (60,000) est divisé en :
    - Train set : 80% (pour l'entraînement)
    - Dev set : 20% (pour la validation et l'early stopping)
    Le test set (10,000) reste séparé pour l'évaluation finale.
    
    Args:
        train_path (str): Chemin vers mnist_train.csv
        test_path (str): Chemin vers mnist_test.csv
        train_split (float): Proportion pour le train set (0.8 = 80%)
        shuffle (bool): Mélanger les données avant le split
    
    Returns:
        tuple: (X_train, Y_train, X_dev, Y_dev, X_test, Y_test)
               Tous les tensors sont sur le device configuré (GPU/CPU)
               Shape: X = (features, batch), Y = (batch,)
    """
    print("\n" + "="*70)
    print("📥 CHARGEMENT DES DONNÉES MNIST")
    print("="*70)
    
    # ========================================================================
    # CHARGEMENT DU TRAIN SET
    # ========================================================================
    
    print("\n🔄 Chargement du train set...")
    data_train_full = pd.read_csv(train_path)
    data_train_full = np.array(data_train_full)
    
    print(f"   ✅ {len(data_train_full):,} exemples chargés")
    
    # Mélanger pour randomiser
    if shuffle:
        np.random.shuffle(data_train_full)
        print(f"   🔀 Données mélangées")
    
    # ========================================================================
    # CHARGEMENT DU TEST SET
    # ========================================================================
    
    print("\n🔄 Chargement du test set...")
    data_test_full = pd.read_csv(test_path)
    data_test_full = np.array(data_test_full)
    
    print(f"   ✅ {len(data_test_full):,} exemples chargés")
    
    # ========================================================================
    # SPLIT TRAIN / DEV
    # ========================================================================
    
    print(f"\n🔪 Split train/dev ({train_split*100:.0f}/{(1-train_split)*100:.0f})...")
    
    split_idx = int(train_split * len(data_train_full))
    data_train = data_train_full[:split_idx]
    data_dev = data_train_full[split_idx:]
    
    print(f"   ✅ Train : {len(data_train):,} exemples")
    print(f"   ✅ Dev   : {len(data_dev):,} exemples")
    
    # ========================================================================
    # CONVERSION EN TENSORS PYTORCH
    # ========================================================================
    
    print(f"\n⚙️  Conversion en tensors PyTorch...")
    
    # Train
    Y_train = torch.tensor(data_train[:, -1], dtype=torch.long).to(device)
    X_train = torch.tensor(data_train[:, :-1].T, dtype=torch.float32).to(device)
    
    # Dev
    Y_dev = torch.tensor(data_dev[:, -1], dtype=torch.long).to(device)
    X_dev = torch.tensor(data_dev[:, :-1].T, dtype=torch.float32).to(device)
    
    # Test
    Y_test = torch.tensor(data_test_full[:, -1], dtype=torch.long).to(device)
    X_test = torch.tensor(data_test_full[:, :-1].T, dtype=torch.float32).to(device)
    
    # ========================================================================
    # RÉSUMÉ
    # ========================================================================
    
    print("\n" + "="*70)
    print("✅ DONNÉES PRÊTES")
    print("="*70)
    
    print(f"\n{'Dataset':<10} {'Samples':<12} {'Shape (X)':<20} {'Shape (Y)':<15} {'Device'}")
    print("-" * 70)
    print(f"{'Train':<10} {X_train.shape[1]:<12,} {str(X_train.shape):<20} {str(Y_train.shape):<15} {X_train.device}")
    print(f"{'Dev':<10} {X_dev.shape[1]:<12,} {str(X_dev.shape):<20} {str(Y_dev.shape):<15} {X_dev.device}")
    print(f"{'Test':<10} {X_test.shape[1]:<12,} {str(X_test.shape):<20} {str(Y_test.shape):<15} {X_test.device}")
    
    print(f"\n📊 Format : X = (features, batch) | Y = (batch,)")
    print(f"📊 Dtype : X = {X_train.dtype} | Y = {Y_train.dtype}")
    print("="*70)
    
    return X_train, Y_train, X_dev, Y_dev, X_test, Y_test


# ============================================================================
# CHARGER LES DONNÉES
# ============================================================================

X_train, Y_train, X_dev, Y_dev, X_test, Y_test = load_mnist_data(
    DATA_TRAIN_PATH, 
    DATA_TEST_PATH,
    train_split=0.8,
    shuffle=True
)


# 🧠 CELLULE 3 : Définition de l'Architecture MLP

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                   ARCHITECTURE MLP (MULTI-LAYER PERCEPTRON)              ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

class MLP(nn.Module):
    """
    Multi-Layer Perceptron pour classification MNIST.
    
    Architecture configurable avec :
    - Nombre de couches cachées variable
    - Taille de chaque couche configurable
    - Choix de la fonction d'activation
    - Dropout pour régularisation
    - Batch Normalization optionnelle
    - Initialisation des poids (He ou Xavier)
    
    Args:
        layer_sizes (list): Liste des tailles de couches cachées
                           Ex: [256, 128, 64] = 3 couches cachées
        activation (str): Fonction d'activation ('relu', 'leaky_relu', 'tanh', 'gelu')
        init_method (str): Méthode d'initialisation ('he', 'xavier')
        dropout (float): Taux de dropout (0.0 = pas de dropout)
        batch_norm (bool): Utiliser la batch normalization
    
    Example:
        >>> model = MLP(layer_sizes=[256, 128], activation='relu', dropout=0.3)
        >>> output = model(X_train)  # Shape: (batch_size, 10)
    """
    
    def __init__(self, layer_sizes, activation='relu', init_method='he',
                 dropout=0.0, batch_norm=False):
        super(MLP, self).__init__()
        
        # ====================================================================
        # CONFIGURATION
        # ====================================================================
        
        self.layer_sizes = layer_sizes
        self.use_batch_norm = batch_norm
        
        # Stockage des couches
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList() if batch_norm else None
        
        # ====================================================================
        # CONSTRUCTION DES COUCHES CACHÉES
        # ====================================================================
        
        input_size = 784  # MNIST = 28×28 pixels
        
        for hidden_size in layer_sizes:
            # Couche linéaire (fully connected)
            self.layers.append(nn.Linear(input_size, hidden_size))
            
            # Batch normalization (optionnelle)
            if batch_norm:
                self.batch_norms.append(nn.BatchNorm1d(hidden_size))
            
            input_size = hidden_size  # Pour la prochaine couche
        
        # ====================================================================
        # COUCHE DE SORTIE
        # ====================================================================
        
        self.output_layer = nn.Linear(input_size, 10)  # 10 classes (0-9)
        
        # ====================================================================
        # RÉGULARISATION
        # ====================================================================
        
        self.dropout = nn.Dropout(dropout)
        
        # ====================================================================
        # FONCTION D'ACTIVATION
        # ====================================================================
        
        activation_dict = {
            'relu': nn.ReLU(),
            'leaky_relu': nn.LeakyReLU(0.01),
            'tanh': nn.Tanh(),
            'gelu': nn.GELU()
        }
        
        if activation not in activation_dict:
            raise ValueError(f"Activation '{activation}' non supportée. "
                           f"Choix: {list(activation_dict.keys())}")
        
        self.activation = activation_dict[activation]
        
        # ====================================================================
        # INITIALISATION DES POIDS
        # ====================================================================
        
        self._initialize_weights(init_method)
    
    
    def _initialize_weights(self, init_method):
        """
        Initialise les poids des couches selon la méthode choisie.
        
        - He (Kaiming) : Recommandé pour ReLU
        - Xavier (Glorot) : Recommandé pour Tanh/Sigmoid
        
        Args:
            init_method (str): 'he' ou 'xavier'
        """
        # Couches cachées
        for layer in self.layers:
            if init_method == 'he':
                nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
            elif init_method == 'xavier':
                nn.init.xavier_normal_(layer.weight)
            else:
                raise ValueError(f"Méthode d'initialisation '{init_method}' non supportée")
            
            nn.init.zeros_(layer.bias)
        
        # Couche de sortie
        if init_method == 'he':
            nn.init.kaiming_normal_(self.output_layer.weight)
        elif init_method == 'xavier':
            nn.init.xavier_normal_(self.output_layer.weight)
        
        nn.init.zeros_(self.output_layer.bias)
    
    
    def forward(self, x):
        """
        Forward pass du réseau.
        
        Args:
            x (Tensor): Input de shape (features, batch) = (784, N)
        
        Returns:
            Tensor: Logits de shape (batch, 10)
        
        Flow:
            (784, N) → Transpose → (N, 784)
            → Linear → BatchNorm → Activation → Dropout
            → ... (répété pour chaque couche cachée)
            → Linear (output) → (N, 10)
        """
        # Transposer pour avoir (batch, features)
        x = x.T  # (features, batch) → (batch, features)
        
        # Passage dans les couches cachées
        for i, layer in enumerate(self.layers):
            x = layer(x)
            
            # Batch normalization (si activée)
            if self.use_batch_norm:
                x = self.batch_norms[i](x)
            
            # Activation
            x = self.activation(x)
            
            # Dropout
            x = self.dropout(x)
        
        # Couche de sortie (pas d'activation, logits bruts)
        x = self.output_layer(x)
        
        return x  # Shape: (batch, 10)


# ============================================================================
# FONCTION D'ÉVALUATION
# ============================================================================

def evaluate_model(model, X, Y):
    """
    Évalue l'accuracy du modèle sur un dataset.
    
    Args:
        model (nn.Module): Modèle à évaluer
        X (Tensor): Features (features, batch)
        Y (Tensor): Labels (batch,)
    
    Returns:
        float: Accuracy (entre 0 et 1)
    """
    model.eval()  # Mode évaluation (désactive dropout)
    
    with torch.no_grad():  # Pas de calcul de gradient
        outputs = model(X)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Classe avec max probabilité
        accuracy = (predicted == Y).float().mean().item()
    
    return accuracy


# ============================================================================
# TEST DE L'ARCHITECTURE
# ============================================================================

print("\n" + "="*70)
print("🧠 TEST DE L'ARCHITECTURE MLP")
print("="*70)

# Créer un modèle test
test_model = MLP(
    layer_sizes=[256, 128, 64],
    activation='relu',
    init_method='he',
    dropout=0.3,
    batch_norm=True
).to(device)

print(f"\n📊 Architecture :")
print(f"   Input : 784 (28×28 pixels)")
print(f"   Couches cachées : [256, 128, 64]")
print(f"   Output : 10 (classes 0-9)")
print(f"   Activation : ReLU")
print(f"   Dropout : 0.3")
print(f"   Batch Norm : True")

# Compter les paramètres
total_params = sum(p.numel() for p in test_model.parameters())
trainable_params = sum(p.numel() for p in test_model.parameters() if p.requires_grad)

print(f"\n🔢 Nombre de paramètres :")
print(f"   Total : {total_params:,}")
print(f"   Entraînables : {trainable_params:,}")

# Test forward pass
print(f"\n⚙️  Test forward pass...")
with torch.no_grad():
    test_output = test_model(X_train[:, :5])  # 5 premiers exemples
    print(f"   Input shape : {X_train[:, :5].shape}")
    print(f"   Output shape : {test_output.shape}")
    print(f"   ✅ Forward pass réussi")

print("\n" + "="*70)
print("✅ ARCHITECTURE MLP PRÊTE")
print("="*70)


# 🎯 CELLULE 4 : Fonction d'Entraînement

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                     FONCTION D'ENTRAÎNEMENT MLP                          ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

def train_model(X, Y, lr, iterations, layer_sizes, activation, init_method,
                l2_lambda, dropout, batch_norm, optimizer_name,
                scheduler_name=None, grad_clip=None):
    """
    Entraîne un modèle MLP avec les hyperparamètres fournis.
    
    Args:
        X (Tensor): Features d'entraînement (features, batch)
        Y (Tensor): Labels (batch,)
        lr (float): Learning rate
        iterations (int): Nombre d'epochs
        layer_sizes (list): Architecture du réseau
        activation (str): Fonction d'activation
        init_method (str): Méthode d'initialisation
        l2_lambda (float): Coefficient de régularisation L2 (weight decay)
        dropout (float): Taux de dropout
        batch_norm (bool): Utiliser batch normalization
        optimizer_name (str): 'adam', 'adamw', ou 'sgd'
        scheduler_name (str): 'plateau' ou None
        grad_clip (float): Valeur max pour gradient clipping (None = pas de clipping)
    
    Returns:
        nn.Module: Modèle entraîné
    """
    
    # ========================================================================
    # CRÉATION DU MODÈLE
    # ========================================================================
    
    model = MLP(
        layer_sizes=layer_sizes,
        activation=activation,
        init_method=init_method,
        dropout=dropout,
        batch_norm=batch_norm
    ).to(device)
    
    # ========================================================================
    # LOSS ET OPTIMISEUR
    # ========================================================================
    
    criterion = nn.CrossEntropyLoss()
    
    # Choix de l'optimiseur
    optimizer_dict = {
        'adam': optuna.Adam(model.parameters(), lr=lr, weight_decay=l2_lambda),
        'adamw': optim.AdamW(model.parameters(), lr=lr, weight_decay=l2_lambda),
        'sgd': optim.SGD(model.parameters(), lr=lr, weight_decay=l2_lambda, momentum=0.9)
    }
    
    if optimizer_name not in optimizer_dict:
        raise ValueError(f"Optimiseur '{optimizer_name}' non supporté. "
                        f"Choix: {list(optimizer_dict.keys())}")
    
    optimizer = optimizer_dict[optimizer_name]
    
    # ========================================================================
    # LEARNING RATE SCHEDULER (OPTIONNEL)
    # ========================================================================
    
    scheduler = None
    if scheduler_name == 'plateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, 
            mode='min',      # Minimiser la loss
            patience=10,     # Attendre 10 epochs avant de réduire lr
            factor=0.5,      # Réduire lr de moitié
            verbose=False
        )
    
    # ========================================================================
    # BOUCLE D'ENTRAÎNEMENT
    # ========================================================================
    
    model.train()  # Mode entraînement
    
    for epoch in range(iterations):
        # Forward pass
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, Y)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping (évite l'explosion des gradients)
        if grad_clip:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        
        # Mise à jour des poids
        optimizer.step()
        
        # Mise à jour du learning rate
        if scheduler and scheduler_name == 'plateau':
            scheduler.step(loss)
    
    return model


print("✅ Fonction d'entraînement définie")


# 🔬 CELLULE 5 : Optimisation Bayésienne avec Optuna

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              OPTIMISATION BAYÉSIENNE DES HYPERPARAMÈTRES                 ║
╚══════════════════════════════════════════════════════════════════════════╝

Objectif : Trouver automatiquement la meilleure configuration du MLP

Hyperparamètres optimisés :
- Architecture : nombre et taille des couches
- Learning rate
- Régularisation L2
- Dropout

Hyperparamètres fixés (déjà optimaux) :
- Activation : ReLU
- Initialisation : He
- Batch Norm : True
- Optimiseur : AdamW
- Scheduler : ReduceLROnPlateau
- Gradient clipping : 0.5
"""

print("\n" + "="*70)
print("🔬 OPTIMISATION BAYÉSIENNE AVEC OPTUNA")
print("="*70)

# ============================================================================
# FONCTION OBJECTIF
# ============================================================================

def optuna_objective(trial):
    """
    Fonction objectif pour Optuna : teste une configuration et retourne l'accuracy.
    
    Args:
        trial (optuna.Trial): Object trial contenant les suggestions d'hyperparamètres
    
    Returns:
        float: Accuracy sur le dev set (à maximiser)
    """
    
    # Suggérer les hyperparamètres
    n_layers = trial.suggest_int('n_layers', 2, 3)  # 2 ou 3 couches cachées
    
    layer_sizes = [
        trial.suggest_int(f'n_neurons_l{i}', 64, 192, step=32)
        for i in range(n_layers)
    ]
    
    lr = trial.suggest_float('learning_rate', 0.001, 0.01, log=True)
    l2_lambda = trial.suggest_float('l2_lambda', 0.005, 0.012, log=True)
    dropout = trial.suggest_float('dropout', 0.5, 0.65)
    
    # Entraîner le modèle
    model = train_model(
        X=X_train,
        Y=Y_train,
        lr=lr,
        iterations=250,  # Moins d'epochs pour aller plus vite
        layer_sizes=layer_sizes,
        activation='relu',
        init_method='he',
        l2_lambda=l2_lambda,
        dropout=dropout,
        batch_norm=True,
        optimizer_name='adamw',
        scheduler_name='plateau',
        grad_clip=0.5
    )
    
    # Évaluer sur le dev set
    dev_accuracy = evaluate_model(model, X_dev, Y_dev)
    
    return dev_accuracy


# ============================================================================
# LANCER L'OPTIMISATION
# ============================================================================

print("\n🚀 Lancement de l'optimisation...")
print(f"   • Nombre de trials : 50")
print(f"   • Sampler : TPE (Tree-structured Parzen Estimator)")
print(f"   • Objectif : Maximiser l'accuracy sur dev set")
print(f"\n⏱️  Temps estimé : 10-15 minutes\n")

# Créer l'étude Optuna
study = optuna.create_study(
    direction='maximize',  # Maximiser l'accuracy
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
)

# Optimiser
study.optimize(
    optuna_objective, 
    n_trials=50,
    show_progress_bar=True
)

# ============================================================================
# RÉSULTATS
# ============================================================================

print("\n" + "="*70)
print("🏆 RÉSULTATS DE L'OPTIMISATION")
print("="*70)

print(f"\n✅ Meilleure accuracy dev : {study.best_value:.4f}")

print(f"\n📋 Meilleurs hyperparamètres :")
for key, value in study.best_params.items():
    print(f"   • {key:<20} : {value}")

# ============================================================================
# SAUVEGARDER LES HYPERPARAMÈTRES
# ============================================================================

print(f"\n💾 Sauvegarde des hyperparamètres...")

best_params = {
    'n_layers': study.best_params['n_layers'],
    'layer_sizes': [
        study.best_params[f'n_neurons_l{i}']
        for i in range(study.best_params['n_layers'])
    ],
    'activation': 'relu',
    'learning_rate': study.best_params['learning_rate'],
    'iterations': 300,  # Plus d'epochs pour l'entraînement final
    'l2_lambda': study.best_params['l2_lambda'],
    'dropout': study.best_params['dropout'],
    'batch_norm': True,
    'init_method': 'he',
    'optimizer': 'adamw',
    'scheduler': 'plateau',
    'grad_clip': 0.5,
    'best_dev_accuracy': study.best_value
}

with open(HYPERPARAMS_PATH, 'w') as f:
    json.dump(best_params, f, indent=2)

print(f"   ✅ Sauvegardé dans : {HYPERPARAMS_PATH}")

print("\n" + "="*70)
print("✅ OPTIMISATION TERMINÉE")
print("="*70)


# 📈 CELLULE 6 : Graphiques Optuna

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              VISUALISATION DES RÉSULTATS OPTUNA                          ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

print("\n📊 Génération des graphiques Optuna...")

# ============================================================================
# GRAPHIQUE 1 : HISTORIQUE D'OPTIMISATION
# ============================================================================

fig1 = plot_optimization_history(study)
fig1.update_layout(
    title="Historique d'Optimisation Bayésienne",
    xaxis_title="Trial",
    yaxis_title="Dev Accuracy",
    width=1200,
    height=600
)
fig1.show()

# ============================================================================
# GRAPHIQUE 2 : IMPORTANCE DES HYPERPARAMÈTRES
# ============================================================================

fig2 = plot_param_importances(study)
fig2.update_layout(
    title="Importance des Hyperparamètres",
    width=1000,
    height=600
)
fig2.show()

print("✅ Graphiques Optuna affichés")


# 🎯 CELLULE 7 : Entraînement Final avec Early Stopping

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              ENTRAÎNEMENT FINAL AVEC EARLY STOPPING                      ║
╚══════════════════════════════════════════════════════════════════════════╝

Utilise les meilleurs hyperparamètres trouvés par Optuna
Implémente l'early stopping pour éviter le surapprentissage
"""

print("\n" + "="*70)
print("🎯 ENTRAÎNEMENT FINAL DU MODÈLE")
print("="*70)

# ============================================================================
# CHARGER LES MEILLEURS HYPERPARAMÈTRES
# ============================================================================

print("\n🔄 Chargement des hyperparamètres optimisés...")

with open(HYPERPARAMS_PATH, 'r') as f:
    best_params = json.load(f)

print(f"✅ Hyperparamètres chargés :")
print(json.dumps(best_params, indent=2))

# ============================================================================
# FONCTION D'ENTRAÎNEMENT AVEC EARLY STOPPING
# ============================================================================

def train_with_early_stopping(X_train, Y_train, X_val, Y_val, hyperparams, 
                               patience=3):
    """
    Entraîne le modèle avec early stopping et learning rate scheduling.
    
    Early Stopping : Arrête l'entraînement si la validation accuracy 
                     ne s'améliore pas pendant 'patience' evaluations.
    
    Args:
        X_train, Y_train: Données d'entraînement
        X_val, Y_val: Données de validation
        hyperparams (dict): Dictionnaire des hyperparamètres
        patience (int): Nombre d'epochs sans amélioration avant arrêt
    
    Returns:
        tuple: (model, best_val_acc, history)
               - model: Meilleur modèle (state du meilleur epoch)
               - best_val_acc: Meilleure accuracy validation
               - history: Historique d'entraînement
    """
    
    # ========================================================================
    # INITIALISATION DU MODÈLE
    # ========================================================================
    
    print("\n🏗️  Construction du modèle...")
    
    model = MLP(
        layer_sizes=hyperparams['layer_sizes'],
        activation=hyperparams['activation'],
        init_method=hyperparams['init_method'],
        dropout=hyperparams['dropout'],
        batch_norm=hyperparams['batch_norm']
    ).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"   ✅ Architecture : {hyperparams['layer_sizes']}")
    print(f"   ✅ Paramètres : {total_params:,}")
    
    # ========================================================================
    # OPTIMISEUR ET SCHEDULER
    # ========================================================================
    
    criterion = nn.CrossEntropyLoss()
    
    optimizer = optim.AdamW(
        model.parameters(),
        lr=hyperparams['learning_rate'],
        weight_decay=hyperparams['l2_lambda']
    )
    
    # Réduire le learning rate si la validation accuracy stagne
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max',      # Maximiser l'accuracy
        patience=5,      # Attendre 5 evaluations
        factor=0.5,      # Diviser lr par 2
        verbose=True     # Afficher les changements de lr
    )
    
    # ========================================================================
    # INITIALISATION DE L'HISTORIQUE
    # ========================================================================
    
    history = {
        'epochs': [],
        'train_losses': [],
        'val_losses': [],
        'train_accs': [],
        'val_accs': []
    }
    
    best_val_acc = 0.0
    patience_counter = 0
    best_model_state = None
    
    # ========================================================================
    # BOUCLE D'ENTRAÎNEMENT
    # ========================================================================
    
    print(f"\n🚀 Début de l'entraînement...")
    print(f"   • Epochs max : {hyperparams.get('iterations', 300)}")
    print(f"   • Early stopping patience : {patience}")
    print(f"   • Evaluation tous les 5 epochs\n")
    
    for epoch in range(hyperparams.get('iterations', 300)):
        # ====================================================================
        # PHASE D'ENTRAÎNEMENT
        # ====================================================================
        
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, Y_train)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        if hyperparams.get('grad_clip'):
            torch.nn.utils.clip_grad_norm_(
                model.parameters(), 
                hyperparams['grad_clip']
            )
        
        # Mise à jour des poids
        optimizer.step()
        
        # ====================================================================
        # ÉVALUATION (tous les 5 epochs)
        # ====================================================================
        
        if epoch % 5 == 0:
            model.eval()
            
            with torch.no_grad():
                # Métriques train
                train_loss = loss.item()
                train_acc = evaluate_model(model, X_train, Y_train)
                
                # Métriques validation
                val_outputs = model(X_val)
                val_loss = criterion(val_outputs, Y_val).item()
                val_acc = evaluate_model(model, X_val, Y_val)
            
            # Mise à jour du scheduler
            scheduler.step(val_acc)
            
            # Sauvegarder l'historique
            history['epochs'].append(epoch)
            history['train_losses'].append(train_loss)
            history['val_losses'].append(val_loss)
            history['train_accs'].append(train_acc)
            history['val_accs'].append(val_acc)
            
            # Affichage
            if epoch % 50 == 0:
                print(f"📊 Epoch {epoch:3d} → "
                      f"Train: {train_acc:.2%} (loss: {train_loss:.4f}) | "
                      f"Val: {val_acc:.2%} (loss: {val_loss:.4f})")
            
            # ================================================================
            # EARLY STOPPING LOGIC
            # ================================================================
            
            if val_acc > best_val_acc:
                # Nouvelle meilleure accuracy → sauvegarder le modèle
                best_val_acc = val_acc
                best_model_state = model.state_dict().copy()
                patience_counter = 0
                
                # Indicateur visuel
                if epoch % 50 == 0:
                    print(f"   ✨ Nouveau meilleur modèle ! Val Acc: {best_val_acc:.4f}")
            
            else:
                # Pas d'amélioration → incrémenter le compteur
                patience_counter += 1
            
            # Arrêt si patience dépassée
            if patience_counter >= patience:
                print(f"\n🛑 Early stopping déclenché à l'epoch {epoch}")
                print(f"   • Meilleure val accuracy : {best_val_acc:.4f}")
                print(f"   • Atteinte à l'epoch : {epoch - patience*5}")
                break
    
    # ========================================================================
    # RESTAURER LE MEILLEUR MODÈLE
    # ========================================================================
    
    if best_model_state:
        model.load_state_dict(best_model_state)
        print(f"\n✅ Meilleur modèle restauré")
    
    return model, best_val_acc, history


# ============================================================================
# ENTRAÎNER LE MODÈLE FINAL
# ============================================================================

model_final, final_val_acc, history = train_with_early_stopping(
    X_train, Y_train, 
    X_dev, Y_dev, 
    best_params, 
    patience=3
)

# ============================================================================
# SAUVEGARDER LE MODÈLE
# ============================================================================

print(f"\n💾 Sauvegarde du modèle...")

torch.save({
    'model_state_dict': model_final.state_dict(),
    'hyperparams': best_params,
    'val_accuracy': final_val_acc,
    'train_history': history
}, MODEL_PATH)

print(f"   ✅ Modèle sauvegardé : {MODEL_PATH}")

print("\n" + "="*70)
print(f"✅ ENTRAÎNEMENT TERMINÉ - Val Accuracy: {final_val_acc:.4f}")
print("="*70)


# 📈 CELLULE 8 : Graphique - Courbes d'Apprentissage

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║            GRAPHIQUE : COURBES D'APPRENTISSAGE (TRAINING HISTORY)        ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

print("\n📊 Génération du graphique : Courbes d'apprentissage")

# ============================================================================
# GRAPHIQUE AVEC DOUBLE AXE Y
# ============================================================================

fig, ax1 = plt.subplots(figsize=(14, 7))

# ========================================================================
# AXE 1 : ACCURACY
# ========================================================================

ax1.plot(history['epochs'], history['train_accs'], 
         label='Train Accuracy', linewidth=2.5, color='#4ECDC4', marker='o', markersize=4)
ax1.plot(history['epochs'], history['val_accs'], 
         label='Val Accuracy', linewidth=2.5, linestyle='--', color='#45B7D1', marker='s', markersize=4)

ax1.set_xlabel('Epochs', fontsize=13, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=13, fontweight='bold', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', labelsize=11)
ax1.tick_params(axis='x', labelsize=11)
ax1.set_ylim([0, 1.05])
ax1.grid(True, alpha=0.3, linestyle='--', linewidth=0.7)
ax1.legend(loc='lower right', fontsize=12, framealpha=0.95)

# ========================================================================
# AXE 2 : LOSS
# ========================================================================

ax2 = ax1.twinx()

ax2.plot(history['epochs'], history['train_losses'], 
         label='Train Loss', linewidth=2.5, linestyle='-', color='#FF6B6B', marker='^', markersize=4)
ax2.plot(history['epochs'], history['val_losses'], 
         label='Val Loss', linewidth=2.5, linestyle='--', color='#EE5A6F', marker='v', markersize=4)

ax2.set_ylabel('Loss', fontsize=13, fontweight='bold', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=11)
ax2.set_ylim([0, max(history['train_losses'] + history['val_losses']) * 1.1])
ax2.legend(loc='upper right', fontsize=12, framealpha=0.95)

# ========================================================================
# TITRE ET ANNOTATIONS
# ========================================================================

plt.title('Courbes d\'Apprentissage - MLP MNIST\n'
          f'Meilleure Val Accuracy: {final_val_acc:.4f}',
          fontsize=16, fontweight='bold', pad=20)

# Annoter le meilleur point
best_epoch_idx = history['val_accs'].index(max(history['val_accs']))
best_epoch = history['epochs'][best_epoch_idx]
best_acc = history['val_accs'][best_epoch_idx]

ax1.annotate(f'Best: {best_acc:.4f}',
             xy=(best_epoch, best_acc),
             xytext=(best_epoch + 20, best_acc - 0.05),
             fontsize=11, fontweight='bold',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
             arrowprops=dict(arrowstyle='->', lw=2, color='black'))

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'mlp_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphique sauvegardé : {GRAPH_DIR / 'mlp_training_history.png'}")


# 🧪 CELLULE 9 : Évaluation sur le Test Set

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                  ÉVALUATION FINALE SUR LE TEST SET                       ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

print("\n" + "="*70)
print("🧪 ÉVALUATION SUR LE TEST SET")
print("="*70)

# ============================================================================
# CHARGER LE MODÈLE FINAL
# ============================================================================

print("\n🔄 Chargement du modèle final...")

checkpoint = torch.load(MODEL_PATH, map_location=device)
best_params = checkpoint['hyperparams']

model_final = MLP(
    layer_sizes=best_params['layer_sizes'],
    activation=best_params['activation'],
    init_method=best_params['init_method'],
    dropout=best_params['dropout'],
    batch_norm=best_params['batch_norm']
).to(device)

model_final.load_state_dict(checkpoint['model_state_dict'])

print(f"✅ Modèle chargé")
print(f"   • Architecture : {best_params['layer_sizes']}")
print(f"   • Val Accuracy : {checkpoint['val_accuracy']:.4f}")

# ============================================================================
# PRÉDICTIONS SUR LE TEST SET
# ============================================================================

print("\n🔄 Génération des prédictions...")

model_final.eval()

with torch.no_grad():
    # Forward pass
    test_outputs = model_final(X_test)
    
    # Probabilités (softmax)
    test_probs = torch.softmax(test_outputs, dim=1)
    
    # Prédictions (classe avec max probabilité)
    _, test_pred = torch.max(test_outputs, 1)
    
    # Convertir en numpy pour sklearn
    y_true = Y_test.cpu().numpy()
    y_pred = test_pred.cpu().numpy()
    y_probs = test_probs.cpu().numpy()

print(f"✅ Prédictions générées pour {len(y_true):,} exemples")

# ============================================================================
# MÉTRIQUES GLOBALES
# ============================================================================

print("\n📊 Calcul des métriques...")

# Accuracy
test_acc = (test_pred == Y_test).float().mean().item()

# AUC-ROC (One-vs-Rest pour multiclasse)
auc_score = roc_auc_score(y_true, y_probs, multi_class='ovr', average='macro')

print("\n" + "="*70)
print("📈 RÉSULTATS FINAUX")
print("="*70)
print(f"\n✅ Test Accuracy : {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"✅ AUC-ROC (macro): {auc_score:.4f}")

# ============================================================================
# RAPPORT DE CLASSIFICATION DÉTAILLÉ
# ============================================================================

print("\n" + "="*70)
print("📊 CLASSIFICATION REPORT (par classe)")
print("="*70)

print(classification_report(y_true, y_pred, digits=4, 
                           target_names=[f'Chiffre {i}' for i in range(10)]))

# ============================================================================
# STATISTIQUES D'ERREURS
# ============================================================================

print("="*70)
print("❌ ANALYSE DES ERREURS")
print("="*70)

# Nombre total d'erreurs
n_errors = (y_true != y_pred).sum()
error_rate = n_errors / len(y_true)

print(f"\n📊 Statistiques globales :")
print(f"   • Total exemples : {len(y_true):,}")
print(f"   • Prédictions correctes : {len(y_true) - n_errors:,} ({(1-error_rate)*100:.2f}%)")
print(f"   • Erreurs : {n_errors:,} ({error_rate*100:.2f}%)")

# Top 5 paires de confusion
from collections import Counter
errors = [(y_true[i], y_pred[i]) for i in range(len(y_true)) if y_true[i] != y_pred[i]]
error_pairs = Counter(errors).most_common(5)

print(f"\n🔝 Top 5 confusions les plus fréquentes :")
for (true_label, pred_label), count in error_pairs:
    pct = (count / n_errors) * 100
    print(f"   {true_label} → {pred_label} : {count:3d} fois ({pct:5.2f}% des erreurs)")

print("="*70)


# 🎨 CELLULE 10 : Graphique - Matrice de Confusion

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              GRAPHIQUE : MATRICE DE CONFUSION                            ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

print("\n📊 Génération de la matrice de confusion...")

# ============================================================================
# CALCULER LA MATRICE DE CONFUSION
# ============================================================================

cm = confusion_matrix(y_true, y_pred)

# Matrice en pourcentages (par ligne = par classe réelle)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# ============================================================================
# VISUALISATION
# ============================================================================

fig, ax = plt.subplots(figsize=(12, 10))

# Heatmap
im = ax.imshow(cm, cmap='Blues', interpolation='nearest')

# Titre
ax.set_title('Matrice de Confusion - Test Set\n'
             f'Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)',
             fontsize=16, fontweight='bold', pad=20)

# Colorbar
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Nombre de prédictions', fontsize=12, fontweight='bold')

# Axes
tick_marks = np.arange(10)
ax.set_xticks(tick_marks)
ax.set_yticks(tick_marks)
ax.set_xticklabels(range(10), fontsize=11)
ax.set_yticklabels(range(10), fontsize=11)

ax.set_xlabel('Chiffre Prédit', fontsize=13, fontweight='bold')
ax.set_ylabel('Chiffre Réel', fontsize=13, fontweight='bold')

# ============================================================================
# ANNOTATIONS
# ============================================================================

thresh = cm.max() / 2.

for i in range(10):
    for j in range(10):
        value = cm[i, j]
        percent = cm_percent[i, j]
        
        # Mettre en évidence les erreurs significatives
        if i != j and value >= 10:  # Erreur avec >= 10 cas
            bbox_props = dict(
                boxstyle='round,pad=0.4', 
                facecolor='red', 
                alpha=0.3, 
                edgecolor='darkred',
                linewidth=2
            )
            text_weight = 'bold'
        else:
            bbox_props = None
            text_weight = 'normal'
        
        # Couleur du texte selon le fond
        text_color = 'white' if value > thresh else 'black'
        
        # Afficher count + pourcentage
        ax.text(j, i, f'{value}\n({percent:.1f}%)',
                ha='center', va='center',
                color=text_color, 
                fontsize=10,
                fontweight=text_weight,
                bbox=bbox_props)

# ============================================================================
# LÉGENDE
# ============================================================================

# Ajouter une note explicative
legend_text = (
    "📊 Interprétation :\n"
    "• Diagonale = Prédictions correctes\n"
    "• Cases rouges = Confusions fréquentes (≥10 cas)\n"
    "• Pourcentages = % par rapport au total de la classe réelle"
)

ax.text(0.02, -0.15, legend_text,
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle='round,pad=0.7', facecolor='lightyellow', 
                 alpha=0.9, edgecolor='gray'))

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'mlp_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphique sauvegardé : {GRAPH_DIR / 'mlp_confusion_matrix.png'}")


# 🗺️ CELLULE 11 : Extraction des Embeddings MLP

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              EXTRACTION DES EMBEDDINGS (DERNIÈRE COUCHE)                 ║
╚══════════════════════════════════════════════════════════════════════════╝

Objectif : Capturer les activations de la dernière couche cachée du MLP
Ces embeddings représentent comment le réseau "voit" les images après apprentissage
"""

print("\n" + "="*70)
print("🗺️  EXTRACTION DES EMBEDDINGS MLP")
print("="*70)

# ============================================================================
# VÉRIFIER SI EMBEDDINGS DÉJÀ EN MÉMOIRE
# ============================================================================

try:
    _ = embeddings
    print(f"✅ Embeddings déjà en mémoire - Shape: {embeddings.shape}")

except NameError:
    print("\n🔄 Extraction des embeddings de la dernière couche...")
    
    # ========================================================================
    # CRÉER UN HOOK POUR CAPTURER LES ACTIVATIONS
    # ========================================================================
    
    activations = {}
    
    def get_activation(name):
        """
        Factory function pour créer un hook.
        
        Le hook capture automatiquement la sortie de la couche
        lors du forward pass.
        """
        def hook(model, input, output):
            activations[name] = output.detach()
        return hook
    
    # ========================================================================
    # ATTACHER LE HOOK À LA DERNIÈRE COUCHE CACHÉE
    # ========================================================================
    
    # model_final.layers[-1] = dernière couche cachée (avant output layer)
    target_layer = model_final.layers[-1]
    
    print(f"   • Couche cible : {target_layer}")
    print(f"   • Shape attendue : (batch_size, {best_params['layer_sizes'][-1]})")
    
    hook_handle = target_layer.register_forward_hook(get_activation('embeddings'))
    
    # ========================================================================
    # FORWARD PASS POUR DÉCLENCHER LE HOOK
    # ========================================================================
    
    with torch.no_grad():
        _ = model_final(X_test)
    
    # ========================================================================
    # RÉCUPÉRER LES EMBEDDINGS
    # ========================================================================
    
    embeddings = activations['embeddings'].cpu().numpy()
    y_true = Y_test.cpu().numpy()
    
    # Retirer le hook
    hook_handle.remove()
    
    print(f"\n✅ Embeddings extraits")
    print(f"   • Shape : {embeddings.shape}")
    print(f"   • Dimension : {embeddings.shape[1]}")
    print(f"   • Nombre d'exemples : {embeddings.shape[0]:,}")
    print(f"   • Min : {embeddings.min():.4f}")
    print(f"   • Max : {embeddings.max():.4f}")
    print(f"   • Mean : {embeddings.mean():.4f}")
    print(f"   • Std : {embeddings.std():.4f}")

print("="*70)


# 🔬 CELLULE 12 : Application t-SNE, UMAP, PaCMAP

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║         RÉDUCTION DE DIMENSIONNALITÉ : t-SNE, UMAP, PaCMAP               ║
╚══════════════════════════════════════════════════════════════════════════╝

Objectif : Visualiser les embeddings MLP en 2D
Permet de voir si le réseau a appris à séparer les classes
"""

print("\n" + "="*70)
print("🔬 COMPARAISON DES MÉTHODES DE RÉDUCTION")
print("="*70)

# ============================================================================
# VÉRIFIER ET INSTALLER LES PACKAGES
# ============================================================================

print("\n📦 Vérification des packages...")

# UMAP
try:
    import umap
    print("   ✅ UMAP disponible")
except ImportError:
    print("   📥 Installation de UMAP...")
    import subprocess
    subprocess.run(["pip", "install", "umap-learn", "-q"], check=True)
    import umap
    print("   ✅ UMAP installé")

# PaCMAP
try:
    import pacmap
    print("   ✅ PaCMAP disponible")
except ImportError:
    print("   📥 Installation de PaCMAP...")
    import subprocess
    subprocess.run(["pip", "install", "pacmap", "-q"], check=True)
    import pacmap
    print("   ✅ PaCMAP installé")

# ============================================================================
# APPLICATION DES 3 MÉTHODES
# ============================================================================

print(f"\n" + "="*70)
print("⏳ APPLICATION DES RÉDUCTIONS (estimation : 1-2 minutes)")
print("="*70)

results = {}

# ----------------------------------------------------------------------------
# 1. t-SNE
# ----------------------------------------------------------------------------

print(f"\n🔄 [1/3] t-SNE en cours...")
start = time.time()

tsne = TSNE(
    n_components=2, 
    random_state=RANDOM_SEED, 
    perplexity=30, 
    n_iter=1000, 
    verbose=0
)
embeddings_tsne = tsne.fit_transform(embeddings)
time_tsne = time.time() - start

print(f"   ✅ t-SNE terminé en {time_tsne:.2f}s")
results['t-SNE'] = {'data': embeddings_tsne, 'time': time_tsne}

# ----------------------------------------------------------------------------
# 2. UMAP
# ----------------------------------------------------------------------------

print(f"\n🔄 [2/3] UMAP en cours...")
start = time.time()

reducer_umap = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    random_state=RANDOM_SEED,
    verbose=False
)
embeddings_umap = reducer_umap.fit_transform(embeddings)
time_umap = time.time() - start

print(f"   ✅ UMAP terminé en {time_umap:.2f}s")
results['UMAP'] = {'data': embeddings_umap, 'time': time_umap}

# ----------------------------------------------------------------------------
# 3. PaCMAP
# ----------------------------------------------------------------------------

print(f"\n🔄 [3/3] PaCMAP en cours...")
start = time.time()

reducer_pacmap = pacmap.PaCMAP(
    n_components=2,
    n_neighbors=15,
    MN_ratio=0.5,
    FP_ratio=2.0,
    random_state=RANDOM_SEED,
    verbose=False
)
embeddings_pacmap = reducer_pacmap.fit_transform(embeddings, init="pca")
time_pacmap = time.time() - start

print(f"   ✅ PaCMAP terminé en {time_pacmap:.2f}s")
results['PaCMAP'] = {'data': embeddings_pacmap, 'time': time_pacmap}

# ============================================================================
# CALCUL DES MÉTRIQUES DE CLUSTERING
# ============================================================================

print(f"\n📊 Calcul des métriques de qualité...")

metrics = {}
for name, result in results.items():
    data = result['data']
    
    # Silhouette Score : mesure la séparation des clusters
    sil = silhouette_score(data, y_true)
    
    # Davies-Bouldin Index : mesure la compacité des clusters
    db = davies_bouldin_score(data, y_true)
    
    metrics[name] = {
        'silhouette': sil,
        'davies_bouldin': db,
        'time': result['time']
    }

# ============================================================================
# TABLEAU COMPARATIF
# ============================================================================

print(f"\n" + "="*70)
print("📊 ANALYSE QUANTITATIVE - EMBEDDINGS MLP")
print("="*70)

print(f"\n{'Méthode':<10} {'Silhouette':<15} {'Davies-B':<15} {'Temps (s)':<12} {'Qualité'}")
print("-" * 70)

for method in ['t-SNE', 'UMAP', 'PaCMAP']:
    scores = metrics[method]
    sil = scores['silhouette']
    db = scores['davies_bouldin']
    t = scores['time']
    
    # Évaluation qualitative
    if sil > 0.7:
        quality = "🥇 Excellente"
    elif sil > 0.5:
        quality = "🥈 Très bonne"
    elif sil > 0.4:
        quality = "🥉 Bonne"
    elif sil > 0.3:
        quality = "✓ Correcte"
    else:
        quality = "⚠️  Modérée"
    
    print(f"{method:<10} {sil:>6.4f} {'':8} {db:>6.4f} {'':8} {t:>6.2f} {'':5} {quality}")

# ============================================================================
# CLASSEMENTS
# ============================================================================

best_quality = max(metrics.items(), key=lambda x: x[1]['silhouette'])
fastest = min(metrics.items(), key=lambda x: x[1]['time'])
best_db = min(metrics.items(), key=lambda x: x[1]['davies_bouldin'])

print(f"\n" + "=" * 70)
print("🏆 CLASSEMENTS")
print("=" * 70)
print(f"   Meilleure séparation : {best_quality[0]} (Silhouette = {best_quality[1]['silhouette']:.4f})")
print(f"   Plus rapide          : {fastest[0]} ({fastest[1]['time']:.2f}s)")
print(f"   Meilleur DB Index    : {best_db[0]} ({best_db[1]['davies_bouldin']:.4f})")

# ============================================================================
# RECOMMANDATION
# ============================================================================

print(f"\n" + "=" * 70)
print("💡 RECOMMANDATION")
print("=" * 70)

speedup_umap = time_tsne / time_umap
speedup_pacmap = time_tsne / time_pacmap

print(f"""
✓ t-SNE   : {metrics['t-SNE']['silhouette']:.3f} silhouette, mais {speedup_umap:.1f}× plus lent qu'UMAP
✓ UMAP    : {metrics['UMAP']['silhouette']:.3f} silhouette, excellent compromis vitesse/qualité
✓ PaCMAP  : {metrics['PaCMAP']['silhouette']:.3f} silhouette, {speedup_pacmap:.1f}× plus rapide que t-SNE

Recommandation : {best_quality[0]} offre la meilleure séparation des classes
→ Le MLP a bien appris à séparer les chiffres dans l'espace latent !
""")

print("="*70)


# 🎨 CELLULE 13 : Graphique - Comparaison 1×3

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║        GRAPHIQUE : COMPARAISON t-SNE, UMAP, PaCMAP (1×3)                 ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

print("\n📊 Génération du graphique : Comparaison 1×3")

fig, axes = plt.subplots(1, 3, figsize=(24, 7))
fig.suptitle('Comparaison des Méthodes de Réduction - Embeddings MLP\n'
             'Chaque point = 1 image du test set (10,000 images)',
             fontsize=18, fontweight='bold', y=0.98)

methods_plot = [
    ('t-SNE', embeddings_tsne, axes[0], 'Préserve les voisinages locaux'),
    ('UMAP', embeddings_umap, axes[1], 'Préserve la topologie globale'),
    ('PaCMAP', embeddings_pacmap, axes[2], 'Équilibre local/global')
]

for method_name, data, ax, subtitle in methods_plot:
    # Scatter plot
    scatter = ax.scatter(
        data[:, 0], data[:, 1],
        c=y_true, cmap='tab10', 
        alpha=0.6, s=30,
        edgecolors='black', linewidth=0.2
    )
    
    # Titre et labels
    ax.set_title(f'{method_name} - Embeddings MLP\n{subtitle}',
                 fontsize=14, pad=10, fontweight='bold')
    ax.set_xlabel(f'{method_name} Dimension 1', fontsize=12)
    ax.set_ylabel(f'{method_name} Dimension 2', fontsize=12)
    ax.grid(alpha=0.3, linestyle='--')
    
    # Annotation temps + qualité
    time_text = (f"⏱️ {results[method_name]['time']:.2f}s\n"
                f"📊 Silhouette: {metrics[method_name]['silhouette']:.3f}")
    ax.text(0.02, 0.98, time_text, transform=ax.transAxes,
           fontsize=10, verticalalignment='top', fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                    alpha=0.9, edgecolor='black', linewidth=1.5))
    
    # Colorbar
    cbar = plt.colorbar(scatter, ax=ax, ticks=range(10))
    cbar.set_label('Chiffre', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'mlp_embeddings_comparison_1x3.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphique sauvegardé : {GRAPH_DIR / 'mlp_embeddings_comparison_1x3.png'}")


# 🔢 CELLULE 14 : Graphique - Vue par Classe (3×10)

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║             GRAPHIQUE : VUE PAR CLASSE (grille 3×10)                     ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

print("\n📊 Génération du graphique : Vue par classe (3×10)")

fig, axes = plt.subplots(3, 10, figsize=(28, 10))
fig.suptitle('Vue par Classe - Embeddings MLP\n'
             'Chaque sous-graphe montre où se situent les exemples d\'un chiffre spécifique',
             fontsize=16, fontweight='bold', y=0.998)

methods_data = [
    ('t-SNE', embeddings_tsne),
    ('UMAP', embeddings_umap),
    ('PaCMAP', embeddings_pacmap)
]

for row, (method_name, data) in enumerate(methods_data):
    for digit in range(10):
        ax = axes[row, digit]
        
        # Masque pour le chiffre actuel
        mask_digit = (y_true == digit)
        
        # Background (tous les autres chiffres en gris)
        ax.scatter(data[~mask_digit, 0], data[~mask_digit, 1],
                   c='lightgray', alpha=0.15, s=5)
        
        # Chiffre actuel en couleur
        ax.scatter(data[mask_digit, 0], data[mask_digit, 1],
                   c=f'C{digit}', alpha=0.8, s=20,
                   edgecolors='black', linewidth=0.2)
        
        # Labels
        if digit == 0:
            ax.set_ylabel(f'{method_name}\nMLP',
                         fontsize=12, fontweight='bold',
                         rotation=0, labelpad=40, va='center')
        
        if row == 0:
            ax.set_title(f'Chiffre {digit}',
                        fontsize=11, fontweight='bold')
        
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(alpha=0.2, linestyle='--')

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'mlp_embeddings_per_class_3x10.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphique sauvegardé : {GRAPH_DIR / 'mlp_embeddings_per_class_3x10.png'}")


# ⚡ CELLULE 15 : Graphique - Performance vs Qualité

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║          GRAPHIQUE : COMPROMIS VITESSE vs QUALITÉ                        ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

print("\n📊 Génération du graphique : Compromis vitesse/qualité")

fig, ax = plt.subplots(figsize=(12, 8))

methods_list = ['t-SNE', 'UMAP', 'PaCMAP']
times = [metrics[m]['time'] for m in methods_list]
silhouettes = [metrics[m]['silhouette'] for m in methods_list]
colors = ['#ff7f0e', '#2ca02c', '#d62728']

# Points
for i, method in enumerate(methods_list):
    ax.scatter(times[i], silhouettes[i], s=600, c=colors[i], alpha=0.7,
              edgecolors='black', linewidth=3, zorder=3)
    
    # Annotations
    ax.annotate(f'{method}\n(MLP Embeddings)',
               (times[i], silhouettes[i]),
               fontsize=14, fontweight='bold', ha='center',
               xytext=(0, -60), textcoords='offset points',
               bbox=dict(boxstyle='round,pad=0.7', facecolor='white',
                        edgecolor=colors[i], linewidth=2.5, alpha=0.95),
               arrowprops=dict(arrowstyle='->', lw=2.5, color=colors[i]))

# Configuration
ax.set_xlabel('Temps d\'exécution (secondes)', fontsize=14, fontweight='bold')
ax.set_ylabel('Silhouette Score (qualité de clustering)', fontsize=14, fontweight='bold')
ax.set_title('Compromis Vitesse vs Qualité - Embeddings MLP\n'
             'Plus haut et à gauche = mieux',
             fontsize=18, fontweight='bold', pad=20)
ax.grid(alpha=0.4, linestyle='--', linewidth=1)
ax.set_xlim(left=-1)
ax.set_ylim([0, 1])

# Zones de qualité
ax.axhspan(0.7, 1.0, alpha=0.1, color='green', zorder=0)
ax.axhspan(0.5, 0.7, alpha=0.1, color='orange', zorder=0)
ax.axhspan(0.3, 0.5, alpha=0.1, color='yellow', zorder=0)

# Annotations des zones
ax.text(0.98, 0.85, 'Excellente\n(>0.7)', transform=ax.transAxes,
        fontsize=11, ha='right', color='green', fontweight='bold')
ax.text(0.98, 0.60, 'Très bonne\n(0.5-0.7)', transform=ax.transAxes,
        fontsize=11, ha='right', color='orange', fontweight='bold')
ax.text(0.98, 0.40, 'Bonne\n(0.3-0.5)', transform=ax.transAxes,
        fontsize=11, ha='right', color='goldenrod', fontweight='bold')

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'mlp_speed_quality_tradeoff.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphique sauvegardé : {GRAPH_DIR / 'mlp_speed_quality_tradeoff.png'}")

print("\n" + "="*70)
print("✅ ANALYSE COMPLÈTE TERMINÉE")
print("="*70)
print(f"📁 Tous les graphiques sauvegardés dans : {GRAPH_DIR}")
